In [9]:
import re
import pandas as pd
import numpy as np
from pathlib import Path
from collections import Counter

pd.set_option('display.max_colwidth', -1)

C:\Users\RistoHinno\AppData\Local\Temp\ipykernel_37632\940619647.py:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


## Data

In [2]:
df=pd.read_csv('data/interim/stenos.csv')
df2023=pd.read_csv('data/interim/stenos2023.csv')

In [3]:
df.shape

(213689, 13)

In [4]:
df2023.shape

(47236, 13)

In [6]:
df.year.value_counts()

2021    22656
2016    20406
2017    19997
2020    19135
2010    17255
2014    17082
2018    16466
2012    16351
2015    15225
2022    13242
2011    12972
2019    12130
2013    10772
Name: year, dtype: int64

In [15]:
df2023.year.value_counts()

2022    31676
2023    15560
Name: year, dtype: int64

In [16]:
df=df[df.year!=2022].append(df2023)
df.shape

C:\Users\RistoHinno\AppData\Local\Temp\ipykernel_37632\1817094240.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df[df.year!=2022].append(df2023)


(247683, 13)

In [17]:
df.head(1)

,heading,speaker,link_video,link_steno,index_pk,index_snd,text,year,month,day,time,seconds_from_start,video_link_wo_time
0,10:00 Istungi rakendamine,Aseesimees Siim Kallas,https://www.youtube.com/watch?v=GaFL2Zc4OMg&t=0,https://stenogrammid.riigikogu.ee/201911191000,PKP-25040,SND-559399,"Auväärt kolleegid, auväärt Riigikogu, tere hommikust! Kui te nüüd korraks jälle keskendute meie suurepärase istungi rakendamisele ja räägite omavahelisi jutte edasi pärast, siis on mul suur rõõm avada Riigikogu täiskogu II istungjärgu 9. töönädala teisipäevane istung. Kõigepealt on eelnõude ja arupärimiste üleandmine, kui selleks on soovi. Eelnõude ja arupärimiste üleandmise soovi ma ei näe. Siis teeme palun kohaloleku kontrolli. Kohaloleku kontroll Kohal on 75 Riigikogu liiget, puudub 26. Sellest piisab täielikult, et asuda meie päevakorra arutelu juurde.",2019,11,19,10:00,0,https://www.youtube.com/watch?v=GaFL2Zc4OMg


In [18]:
df.text.isnull().sum()

6317

In [19]:
#remove cases where text is missing - might be some data collection errors,
#but not much compared to overall amount of data (less than 2%)
df=df[~pd.isnull(df.text)]
df.shape

(241366, 13)

## See chars we have - is there something funny

In [20]:
cnt_char=Counter(''.join(df.text).lower())

In [21]:
len(cnt_char)

158

In [22]:
cnt_char.most_common(200)

[(' ', 24400713),
 ('e', 18924027),
 ('a', 17088228),
 ('i', 14892231),
 ('s', 13622917),
 ('t', 11672334),
 ('l', 9130965),
 ('u', 9007766),
 ('n', 7411497),
 ('k', 7058496),
 ('o', 5956143),
 ('m', 5880225),
 ('d', 5637951),
 ('r', 4276758),
 ('v', 3224725),
 ('g', 2941112),
 ('p', 2375498),
 ('j', 2359137),
 ('h', 2333069),
 (',', 2311920),
 ('ä', 2056157),
 ('õ', 1761167),
 ('.', 1594811),
 ('ü', 1039977),
 ('b', 1011381),
 ('ö', 416892),
 ('!', 341567),
 ('\xa0', 319491),
 ('0', 219697),
 ('f', 169592),
 ('1', 164217),
 ('-', 159462),
 ('2', 148869),
 ('?', 123519),
 ('–', 85216),
 ('5', 68026),
 ('3', 59229),
 ('"', 57323),
 (':', 52576),
 ('4', 49329),
 ('6', 42252),
 ('7', 41100),
 ('9', 40711),
 ('8', 37162),
 ('%', 36779),
 ('c', 22125),
 ('[', 21258),
 (']', 21258),
 (')', 14607),
 ('(', 14231),
 ('‑', 11775),
 ('§', 9967),
 ('y', 6223),
 ('š', 5266),
 ('x', 3532),
 ('ž', 3512),
 ('w', 3364),
 ('z', 2685),
 (';', 2675),
 ('/', 2131),
 ("'", 2067),
 ('+', 1360),
 ('…', 1210),

## Make text without punctuation (migth be useful for topic modelling)
## Also remove some junk

In [23]:
def remove_punct(text):
    """remove punctuation, junk from text
    -INPUT:
        -text: str of text to be cleaned
    -OUTPUT:
        -cleaned text str"""
    text=re.sub('(?<=[^0-9])\. ', ' ', text) #keep . if they are between characters (no space) or after numbers
    text_clean=text.lower().replace('\x98', ' ').replace('\x9f', ' ').replace('\x84', ' ').replace('\x9b', ' ')\
    .replace('\xad', ' ')\
    .replace('\xa0', ' ').replace('...', ' ')\
    .replace('!',' ').replace('\?',' ').replace('?',' ').replace(', ',' ').replace(')',' ')\
    .replace('(',' ').replace('"',' ')\
    .replace('ºC', ' kraadi ').replace(' – ', ' ').replace(':', ' ').replace('–', ' ').\
    replace('[---]', ' ').replace('[', '').replace(']', '')\
    .replace(';', ' ').replace('+', ' pluss ').replace(' & ', '&').replace('”', ' ').\
    replace('„', ' ').replace('=', ' võrdub ')\
    .replace('´', "'").replace('’', "'").replace('“', ' ').replace(' ±', ' pluss-miinus ').\
    replace(' ..',' ').replace('`', "'")\
    .replace('…', ' ').replace('″', ' ').replace(' = ', ' võrdub ')
    text_clean=' '.join(text_clean.split())
    if text_clean.endswith('.'):
        text_clean=text_clean[:-1]
    return text_clean

remove_punct('Austatud Riigikogu, tere päevast! Teate mis, ')

'austatud riigikogu tere päevast teate mis'

In [24]:
df['text_wo_punct']=df.text.apply(remove_punct)

In [25]:
#we should have less funny characters now
cnt_char_clean=Counter(''.join(df.text_wo_punct).lower())
print(len(cnt_char_clean))
cnt_char_clean.most_common(200)

129


[(' ', 24415246),
 ('e', 18924027),
 ('a', 17088228),
 ('i', 14892247),
 ('s', 13625661),
 ('t', 11672334),
 ('l', 9132333),
 ('u', 9009158),
 ('n', 7411505),
 ('k', 7058496),
 ('o', 5956143),
 ('m', 5880233),
 ('d', 5637967),
 ('r', 4276774),
 ('v', 3224741),
 ('g', 2941112),
 ('p', 2376866),
 ('j', 2359137),
 ('h', 2333069),
 ('ä', 2056157),
 ('õ', 1761183),
 ('ü', 1039977),
 ('b', 1011397),
 ('ö', 416892),
 ('0', 219697),
 ('f', 169592),
 ('1', 164217),
 ('-', 158633),
 ('2', 148869),
 ('.', 148026),
 ('5', 68026),
 ('3', 59229),
 ('4', 49329),
 ('6', 42252),
 ('7', 41100),
 ('9', 40711),
 ('8', 37162),
 ('%', 36779),
 ('c', 22125),
 (',', 19763),
 ('‑', 11775),
 ('§', 9967),
 ('y', 6223),
 ('š', 5266),
 ('x', 3532),
 ('ž', 3512),
 ('w', 3364),
 ('z', 2685),
 ("'", 2165),
 ('/', 2131),
 ('q', 359),
 ('&', 196),
 ('é', 145),
 ('à', 116),
 ('á', 82),
 ('č', 45),
 ('å', 29),
 ('×', 27),
 ('è', 25),
 ('р', 23),
 ('с', 20),
 ('ó', 19),
 ('и', 19),
 ('°', 16),
 ('@', 14),
 ('ï', 14),
 ('í

## Remove "v a h e a e g"

In [26]:
#not really said but indicates when vaheaeg started
df['text_wo_punct']=df.text_wo_punct.str.replace('v a h e a e g', '')
df['text_wo_punct']=df.text_wo_punct.str.replace('v a h e a eg', '')

## Save data

In [27]:
df.to_csv('data/interim/stenos_cleaned.csv', index=False)